<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Query Service</b>
</header>

<p style = 'font-size:16px;font-family:Arial'>Teradata® Query Service is a middleware that provides REST APIs for connecting to Teradata Vantage. Query Service allows you to query a Teradata-supported database from web pages, mobile devices, or scripting language using HTTP as the wire protocol and JSON as the data interchange format. Applications can use Query Service to access a Teradata-supported database without a driver.Query Service provides APIs to:
<ul style = 'font-size:16px;font-family:Arial'> 
    <li>Configure Teradata-supported systems</li>
    <li>Submit SQL queries and access responses</li>
    <li>Create database sessions</li>
    <li>Access database and object metadata</li>
    </ul>
<p style = 'font-size:16px;font-family:Arial'>For more information check  <a href = 'https://docs.teradata.com/search/all?query=Teradata+Query+Service&filters=prodname~%2522Teradata+Query+Service%2522&sort=last_update&content-lang=en-US'>here.</a>  <br>
 Query Service is a suitable tool in cases when working with a Teradata driver is not practical or feasible, some <ul style = 'font-size:16px;font-family:Arial'> <li>A web or mobile application that communicates with services that expose their own APIs.
     This projects illustrates this specific use case.</li><li>Working with Low Code tools that allow HTTP calls but not the installation of libraries.</li><li>Execution of SQL Queries from the browser.Teradata web-based SQL editors</li></ul>
     <p style = 'font-size:16px;font-family:Arial'>In this demo we will see common examples to help you get started with Query Service API. The examples in this document use Python, and you can use these to create examples in your language of choice.
<hr>

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Connect to Vantage</h1>

In [ ]:
# import libraries
from teradataml import *

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then <b>use down arrow</b> to go to next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username = 'demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Query_Service.ipynb;' UPDATE FOR SESSION;''')

<h2 style = 'font-size:20px;font-family:Arial;color:#E37C4D'>Getting Data for This Demo</h2>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage. There are two statements in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.<br>We will use query this table from Query Service.

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_Telco_cloud');"    # Takes about 30 seconds
# %run -i ../run_procedure.py "call get_data('DEMO_Telco_local');"    # Takes about 30 seconds

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Prerequisites for Query Service</h1>
<p style = 'font-size:16px;font-family:Arial;'>For the Query Service API's to work we need the following
<ul style = 'font-size:16px;font-family:Arial'><li>Access to a VantageCloud system where Query Service is provisioned, or a VantageCore with Query Service enabled connectivity.</li> <li>Query Service hostname and system name </li><li>Authorization credentials to connect to the database</li>
</ul>

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Getting Query Service hostname and system name </h1>
<p style = 'font-size:16px;font-family:Arial'><b>QS_HOSTNAME</b>: Server where Query Service is installed. For our demo the hostname can be found from Clearscape Analytics dashboard.<br>Teradata Query Service uses the ports:<li style = 'font-size:16px;font-family:Arial'>1080 - HTTP</li><li style = 'font-size:16px;font-family:Arial'>1443 - HTTPS</li>
<p style = 'font-size:16px;font-family:Arial'>Thus for our demo the URL will be https:// &lt;QS_HOSTNAME&gt;:1443/<br><b>SYSTEM_NAME</b>: Preconfigured alias of the system. In CSAE the system name is 'local'<br><br><b>Please replace these values with your own system values.</b>


<h2 style = 'font-size:20px;font-family:Arial;color:#E37C4D'>Connect to Query Service instance </h2>
<p style = 'font-size:16px;font-family:Arial'>Provide valid credentials to access the target Analytics Database using HTTP Basic or JWT authentication.

<h2 style = 'font-size:20px;font-family:Arial;color:#E37C4D'>HTTP Basic authentication</h2>
<p style = 'font-size:16px;font-family:Arial'>The database username and password are combined into a string ("username : password") which is then encoded using Base64. The API response contains the authorization method and encoded credentials.

In [ ]:
import requests
import json
import base64
import getpass
requests.packages.urllib3.disable_warnings()

# run it from local.
# please update the db_url with your hostname
db_url = input("Enter the hostname:")
db_user = 'demo_user'
db_password = password 


auth_encoded = db_user + ':' + db_password
auth_encoded = base64.b64encode(bytes(auth_encoded, 'utf-8'))
auth_str = 'Basic ' + auth_encoded.decode('utf-8')

print(auth_str)

headers_https = {
  'Content-Type': 'application/json',
  'Authorization': auth_str # base 64 encoded username:password
}

print(headers_https)

<h2 style = 'font-size:20px;font-family:Arial;color:#E37C4D'>JWT authentication</h2>
<p style = 'font-size:16px;font-family:Arial'> Prerequisites for JWT authentication is that the user must already exist in the database and the database must be JWT enabled.

In [ ]:
auth_encoded_jwt = "<YOUR_JWT_HERE>"
auth_str = "Bearer " + auth_encoded_jwt

headers_jwt = {
  'Content-Type': 'application/json',
  'Authorization': auth_str
}

print(headers_jwt)

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Make a simple API request with basic options </h1>
<p style = 'font-size:16px;font-family:Arial'>Let's submit a simple query to the system using our connection token.<br>
<br>We will run the below query to the system with the alias <b>SYSTEM_NAME</b><br>
`SELECT * FROM DBC.DBCInfo;
`
<ul style = 'font-size:16px;font-family:Arial'><li><b>'format': 'OBJECT' </b>is The format for response. The formats supported are: JSON object, JSON array, and CSV.<br>The JSON object format creates one JSON object per row where the column name is the field name, and the column value is the field value.</li><li><b>'includeColumns': true </b>is the request to include column metadata, such as column names and types, in the response.</li><li><b>'rowLimit': 4 </b>is The number of rows to be returned from a query.</li></ul>

In [ ]:
qs_url = 'https://' + db_url + ':1443/systems/local/queries'

payload = {
  'query': 'SELECT CustomerID,Gender,SeniorCitizen,Partner,Dependents FROM DEMO_Telco.Customer_Churn;',
  'format': 'OBJECT',
  'includeColumns': True,
  'rowLimit': 4
}

payload_json = json.dumps(payload)

response = requests.request('POST', qs_url, headers = headers_https, data = payload_json, verify = False)

num_rows = response.json().get('results')[0].get('rowCount')
print('NUMBER of ROWS', num_rows)
print('==========================================================')

print(response.json())

<h2 style = 'font-size:20px;font-family:Arial;color:#E37C4D'>Request a response in CSV format</h2>
<p style = 'font-size:16px;font-family:Arial'>    To return an API response in CSV format, set the format field in the request with the value <b>CSV</b>.<br>The CSV format contains only the query results and not response metadata. The response contains a line for each row, where each line contains the row columns separated by a comma. The following example returns the data as comma-separated values.

In [ ]:
# CSV with all rows included
qs_url = 'https://' + db_url + ':1443/systems/local/queries'

payload = {
  'query': 'SELECT CustomerID,Gender,SeniorCitizen,Partner,Dependents FROM DEMO_Telco.Customer_Churn;',
  'format': 'CSV',
  'includeColumns': True,
  'rowLimit': 4
}

payload_json = json.dumps(payload)

response = requests.request('POST', qs_url, headers = headers_https, data = payload_json, verify = False)
print(response.text)

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Use explicit session to submit a query </h1>
<p style = 'font-size:16px;font-family:Arial'>Use explicit sessions when a transaction needs to span multiple requests or when using volatile tables. These sessions are only reused if you reference the sessions in a query request. The request is queued if a request references an explicit session already in use.<br><b>Create a session</b><br>Send a POST request to the /system/&lt;SYSTEM_NAME&gt;/sessions endpoint. The request creates a new database session and returns the session details as the response.

In [ ]:
# first create a session
qs_url = 'https://' + db_url + ':1443/systems/local/sessions'

payload = {
  'auto_commit': True
}

payload_json = json.dumps(payload)

response = requests.request('POST', qs_url, headers = headers_https, data = payload_json, verify = False)

print(response.text)

<p style = 'font-size:16px;font-family:Arial'>In the example above the request includes 'auto_commit': True - the request to commit the query upon completion.<br> Use the session created in Step 1(above) to submit queries below.<br>Send a POST request to the /system/&lt;SYSTEM_NAME&gt;/queries endpoint.The request submits queries to the target system and returns the release and version number of the target system.<br>We will run the below query to the system with the alias <b>SYSTEM_NAME</b><br>
`SELECT * FROM DBC.DBCInfo;
`
<ul style = 'font-size:16px;font-family:Arial'><li><b>'format': 'OBJECT' </b>is The format for response.</li><li><b>'Session': &lt;Session ID&gt; </b>is the session ID returned in Step 1 to create an explicit session.    

In [ ]:
sid = input("Enter the session id from returned from above code:")

# use this session to submit queries afterwards

qs_url = 'https://' + db_url + ':1443/systems/local/queries'

payload = {
  'query': 'SELECT CustomerID,Gender,SeniorCitizen,Partner,Dependents FROM DEMO_Telco.Customer_Churn;',
  'format': 'OBJECT',
  'rowLimit': 4,
  'session': sid # <-- sessionId
}
payload_json = json.dumps(payload)

response = requests.request('POST', qs_url, headers = headers_https, data = payload_json, verify = False)

print(response.text)

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Use asynchronous queries </h1>
<p style = 'font-size:16px;font-family:Arial'>Use asynchronous queries when a system or network performance is affected by querying a large group of data or long running queries.<br>Submit asynchronous queries to the target system and retrieve a Query ID.<br>Send a POST request to the /system/&lt;SYSTEM_NAME&gt;/queries endpoint.<br>We will run the below query to the system with the alias <b>SYSTEM_NAME</b><br>
`SELECT * FROM DBC.DBCInfo;
`
<ul style = 'font-size:16px;font-family:Arial'><li><b>'format': 'OBJECT' </b>is The format for response.</li><li><b>'spooled_result_set': True </b>is the indication that the request is asynchronous.

In [ ]:
## Run asynchronous query .

qs_url = 'https://' + db_url + ':1443/systems/local/queries'

payload = {
  'query': 'SELECT CustomerID,Gender,SeniorCitizen,Partner,Dependents FROM DEMO_Telco.Customer_Churn sample 100;',
  'format': 'OBJECT',
  'spooled_result_set': True
}

payload_json = json.dumps(payload)
response = requests.request('POST', qs_url, headers = headers_https, data = payload_json, verify = False)

print(response.text)

<p style = 'font-size:16px;font-family:Arial'>Get query details using the ID retrieved from Step 1(above)<br>Send a GET request to the /system/&lt;SYSTEM_NAME&gt;/queries/&lt;queryID&gt; endpoint, replacing &lt;queryID&gt; with the ID retrieved from Step 1.<br>The request returns the details of the specific query, including queryState, queueOrder, queueDuration, and so on.  

In [ ]:
aqid = input("Enter the Query id from returned from above code (asynchronous queries):")
#update the queryID below
qs_url = 'https://' + db_url + ':1443/systems/local/queries/' + aqid

payload_json = json.dumps(payload)
response = requests.request('GET', qs_url, headers = headers_https, verify = False)

print(response.text)

<h2 style = 'font-size:20px;font-family:Arial;color:#E37C4D'>View resultset for asynchronous query</h2>
<p style = 'font-size:16px;font-family:Arial'>
Send a GET request to the /system/&lt;SYSTEM_NAME&gt;/queries/&lt;queryID&gt;/results endpoint, replacing &lt;queryID&gt; with the ID retrieved from Step 1. The request returns an array of the result sets and update counts produced by the submitted query.

In [ ]:
#update the queryID below
qs_url = 'https://' + db_url + ':1443/systems/local/queries/' + aqid + '/results'

payload_json = json.dumps(payload)
response = requests.request('GET', qs_url, headers = headers_https, verify = False)

print(response.text)

<h2 style = 'font-size:20px;font-family:Arial;color:#E37C4D'>Get a list of active or queued queries</h2>
<p style = 'font-size:16px;font-family:Arial'>Send a GET request to the /system/&lt;SYSTEM_NAME&gt;/queries endpoint. The request returns the IDs of active queries.

In [ ]:
qs_url = 'https://' + db_url + ':1443/systems/local/queries/'

payload = {}

response = requests.request('GET', qs_url, headers=headers_https, data=payload, verify=False)

print(response.json())

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Conclusion</b></h1>

<p style = 'font-size:16px;font-family:Arial'>In this demo we have seen how we can use Teradata Query Service REST APIs to  query the Vantage system and integrate with the client APIs.

<hr>
<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>Cleanup</b></h1>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_Telco');"    # Takes 5 seconds

In [ ]:
remove_context()

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>